<a href="https://colab.research.google.com/github/orlovaea/nlp_ha/blob/main/nlp_ha9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 10. Генерация текста

### Задание 1 (8 баллов).

Попробуйте дообучать GPT на каком-то другом тексте (можете попробовать любые стихи или какие-то специфичные вещи вроде анекдотов, теорий заговоров, постов в помоечных телеграм каналах, текстов журналистов и СМИ с выразительным стилем). 
Попробуйте разные методы и параметры генерации (beam search, температура, top_k и тп). Сохраните в тетрадке несколько хороших сгенерированных текстов


In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

In [3]:
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path, use_cache=False).to(DEVICE)

In [4]:
# Маяковский, Облако в штанах
text = """
Вашу мысль,
мечтающую на размягченном мозгу,
как выжиревший лакей на засаленной кушетке,
буду дразнить об окровавленный сердца лоскут:
досыта изъиздеваюсь, нахальный и едкий.

У меня в душе ни одного седого волоса,
и старческой нежности нет в ней!
Мир огромив мощью голоса,
иду — красивый,
двадцатидвухлетний.

Нежные!
Вы любовь на скрипки ложите.
Любовь на литавры ложит грубый.
А себя, как я, вывернуть не можете,
чтобы были одни сплошные губы!

Приходите учиться —
из гостиной батистовая,
чинная чиновница ангельской лиги.

И которая губы спокойно перелистывает,
как кухарка страницы поваренной книги.

Хотите —
буду от мяса бешеный
— и, как небо, меняя тона —
хотите —
буду безукоризненно нежный,
не мужчина, а — облако в штанах!

Не верю, что есть цветочная Ницца!
Мною опять славословятся
мужчины, залежанные, как больница,
и женщины, истрепанные, как пословица.


1

Вы думаете, это бредит малярия?

Это было,
было в Одессе.

«Приду в четыре»,— сказала Мария.
Восемь.
Девять.
Десять.

Вот и вечер
в ночную жуть
ушел от окон,
хмурый,
декабрый.

В дряхлую спину хохочут и ржут
канделябры.

Меня сейчас узнать не могли бы:
жилистая громадина
стонет,
корчится.
Что может хотеться этакой глыбе?
А глыбе многое хочется!

Ведь для себя не важно
и то, что бронзовый,
и то, что сердце — холодной железкою.
Ночью хочется звон свой
спрятать в мягкое,
в женское.

И вот,
громадный,
горблюсь в окне,
плавлю лбом стекло окошечное.
Будет любовь или нет?
Какая —
большая или крошечная?
Откуда большая у тела такого:
должно быть, маленький,
смирный любёночек.
Она шарахается автомобильных гудков.
Любит звоночки коночек.

Еще и еще,
уткнувшись дождю
лицом в его лицо рябое,
жду,
обрызганный громом городского прибоя.

Полночь, с ножом мечась,
догнала,
зарезала,—
вон его!

Упал двенадцатый час,
как с плахи голова казненного.

В стеклах дождинки серые
свылись,
гримасу громадили,
как будто воют химеры
Собора Парижской Богоматери.

Проклятая!
Что же, и этого не хватит?
Скоро криком издерется рот.
Слышу:
тихо,
как больной с кровати,
спрыгнул нерв.
И вот,—
сначала прошелся
едва-едва,
потом забегал,
взволнованный,
четкий.
Теперь и он и новые два
мечутся отчаянной чечеткой.

Рухнула штукатурка в нижнем этаже.

Нервы —
большие,
маленькие,
многие!—
скачут бешеные,
и уже
у нервов подкашиваются ноги!

А ночь по комнате тинится и тинится,—
из тины не вытянуться отяжелевшему глазу.

Двери вдруг заляскали,
будто у гостиницы
не попадает зуб на зуб.

Вошла ты,
резкая, как «нате!»,
муча перчатки замш,
сказала:
«Знаете —
я выхожу замуж».

Что ж, выходите.
Ничего.
Покреплюсь.
Видите — спокоен как!
Как пульс
покойника.
Помните?
Вы говорили:
«Джек Лондон,
деньги,
любовь,
страсть»,—
а я одно видел:
вы — Джоконда,
которую надо украсть!
И украли.

Опять влюбленный выйду в игры,
огнем озаряя бровей загиб.
Что же!
И в доме, который выгорел,
иногда живут бездомные бродяги!

Дразните?
«Меньше, чем у нищего копеек,
у вас изумрудов безумий».
Помните!
Погибла Помпея,
когда раздразнили Везувий!

Эй!
Господа!
Любители
святотатств,
преступлений,
боен,—
а самое страшное
видели —
лицо мое,
когда
я
абсолютно спокоен?

И чувствую —
«я»
для меня мало.
Кто-то из меня вырывается упрямо.

Allo!
Кто говорит?
Мама?
Мама!
Ваш сын прекрасно болен!
Мама!
У него пожар сердца.
Скажите сестрам, Люде и Оле,—
ему уже некуда деться.
Каждое слово,
даже шутка,
которые изрыгает обгорающим ртом он,
выбрасывается, как голая проститутка
из горящего публичного дома.
Люди нюхают —
запахло жареным!
Нагнали каких-то.
Блестящие!
В касках!
Нельзя сапожища!
Скажите пожарным:
на сердце горящее лезут в ласках.
Я сам.
Глаза наслезнённые бочками выкачу.
Дайте о ребра опереться.
Выскочу! Выскочу! Выскочу! Выскочу!
Рухнули.
Не выскочишь из сердца!

На лице обгорающем
из трещины губ
обугленный поцелуишко броситься вырос.

Мама!
Петь не могу.
У церковки сердца занимается клирос!

Обгорелые фигурки слов и чисел
из черепа,
как дети из горящего здания.
Так страх
схватиться за небо
высил
горящие руки «Лузитании».

Трясущимся людям
в квартирное тихо
стоглазое зарево рвется с пристани.
Крик последний,—
ты хоть
о том, что горю, в столетия выстони!


2

Славьте меня!
Я великим не чета.
Я над всем, что сделано,
ставлю «nihil».

Никогда
ничего не хочу читать.
Книги?
Что книги!

Я раньше думал —
книги делаются так:
пришел поэт,
легко разжал уста,
и сразу запел вдохновенный простак —
пожалуйста!
А оказывается —
прежде чем начнет петься,
долго ходят, размозолев от брожения,
и тихо барахтается в тине сердца
глупая вобла воображения.
Пока выкипячивают, рифмами пиликая,
из любвей и соловьев какое-то варево,
улица корчится безъязыкая —
ей нечем кричать и разговаривать.

Городов вавилонские башни,
возгордясь, возносим снова,
а бог
города на пашни
рушит,
мешая слово.

Улица муку молча пёрла.
Крик торчком стоял из глотки.
Топорщились, застрявшие поперек горла,
пухлые taxi и костлявые пролетки
грудь испешеходили.

Чахотки площе.
Город дорогу мраком запер.

И когда —
все-таки!—
выхаркнула давку на площадь,
спихнув наступившую на горло паперть,
думалось:
в хорах архангелова хорала
бог, ограбленный, идет карать!

А улица присела и заорала:
«Идемте жрать!»

Гримируют городу Круппы и Круппики
грозящих бровей морщь,
а во рту
умерших слов разлагаются трупики,
только два живут, жирея —
«сволочь»
и еще какое-то,
кажется, «борщ».

Поэты,
размокшие в плаче и всхлипе,
бросились от улицы, ероша космы:
«Как двумя такими выпеть
и барышню,
и любовь,
и цветочек под росами?»
А за поэтами —
уличные тыщи:
студенты,
проститутки,
подрядчики.

Господа!
Остановитесь!
Вы не нищие,
вы не смеете просить подачки!

Нам, здоровенным,
с шаго саженьим,
надо не слушать, а рвать их —
их,
присосавшихся бесплатным приложением
к каждой двуспальной кровати!

Их ли смиренно просить:
«Помоги мне!»
Молить о гимне,
об оратории!
Мы сами творцы в горящем гимне —
шуме фабрики и лаборатории.

Что мне до Фауста,
феерией ракет
скользящего с Мефистофелем в небесном паркете!
Я знаю —
гвоздь у меня в сапоге
кошмарней, чем фантазия у Гете!

Я,
златоустейший,
чье каждое слово
душу новородит,
именинит тело,
говорю вам:
мельчайшая пылинка живого
ценнее всего, что я сделаю и сделал!

Слушайте!
Проповедует,
мечась и стеня,
сегодняшнего дня крикогубый Заратустра!
Мы
с лицом, как заспанная простыня,
с губами, обвисшими, как люстра,
мы,
каторжане города-лепрозория,
где золото и грязь изъязвили проказу,—
мы чище венецианского лазорья,
морями и солнцами омытого сразу!

Плевать, что нет
у Гомеров и Овидиев
людей, как мы,
от копоти в оспе.
Я знаю —
солнце померкло б, увидев
наших душ золотые россыпи!

Жилы и мускулы — молитв верней.
Нам ли вымаливать милостей времени!
Мы —
каждый —
держим в своей пятерне
миров приводные ремни!

Это взвело на Голгофы аудиторий
Петрограда, Москвы, Одессы, Киева,
и не было ни одного,
который
не кричал бы:
«Распни,
распни его!»
Но мне —
люди,
и те, что обидели —
вы мне всего дороже и ближе.

Видели,
как собака бьющую руку лижет?!

Я,
обсмеянный у сегодняшнего племени,
как длинный
скабрезный анекдот,
вижу идущего через горы времени,
которого не видит никто.

Где глаз людей обрывается куцый,
главой голодных орд,
в терновом венце революций
грядет шестнадцатый год.

А я у вас — его предтеча;
я — где боль, везде;
на каждой капле слёзовой течи
распял себя на кресте.
Уже ничего простить нельзя.
Я выжег души, где нежность растили.
Это труднее, чем взять
тысячу тысяч Бастилий!

И когда,
приход его
мятежом оглашая,
выйдете к спасителю —
вам я
душу вытащу,
растопчу,
чтоб большая!—
и окровавленную дам, как знамя.


3

Ах, зачем это,
откуда это
в светлое весело
грязных кулачищ замах!

Пришла
и голову отчаянием занавесила
мысль о сумасшедших домах.

И —
как в гибель дредноута
от душащих спазм
бросаются в разинутый люк —
сквозь свой
до крика разодранный глаз
лез, обезумев, Бурлюк.
Почти окровавив исслезенные веки,
вылез,
встал,
пошел
и с нежностью, неожиданной в жирном человеке
взял и сказал:
«Хорошо!»
Хорошо, когда в желтую кофту
душа от осмотров укутана!
Хорошо,
когда брошенный в зубы эшафоту,
крикнуть:
«Пейте какао Ван-Гутена!»

И эту секунду,
бенгальскую,
громкую,
я ни на что б не выменял,
я ни на...

А из сигарного дыма
ликерною рюмкой
вытягивалось пропитое лицо Северянина.
Как вы смеете называться поэтом
и, серенький, чирикать, как перепел!
Сегодня
надо
кастетом
кроиться миру в черепе!



Вы,
обеспокоенные мыслью одной —
«изящно пляшу ли»,—
смотрите, как развлекаюсь
я —
площадной
сутенер и карточный шулер.
От вас,
которые влюбленностью мокли,
от которых
в столетия слеза лилась,
уйду я,
солнце моноклем
вставлю в широко растопыренный глаз.

Невероятно себя нарядив,
пойду по земле,
чтоб нравился и жегся,
а впереди
на цепочке Наполеона поведу, как мопса.
Вся земля поляжет женщиной,
заерзает мясами, хотя отдаться;
вещи оживут —
губы вещины
засюсюкают:
«цаца, цаца, цаца!»

Вдруг
и тучи
и облачное прочее
подняло на небе невероятную качку,
как будто расходятся белые рабочие,
небу объявив озлобленную стачку.
Гром из-за тучи, зверея, вылез,
громадные ноздри задорно высморкая,
и небье лицо секунду кривилось
суровой гримасой железного Бисмарка.
И кто-то,
запутавшись в облачных путах,
вытянул руки к кафе —
и будто по-женски,
и нежный как будто,
и будто бы пушки лафет.

Вы думаете —
это солнце нежненько
треплет по щечке кафе?
Это опять расстрелять мятежников
грядет генерал Галифе!

Выньте, гулящие, руки из брюк —
берите камень, нож или бомбу,
а если у которого нету рук —
пришел чтоб и бился лбом бы!
Идите, голодненькие,
потненькие,
покорненькие,
закисшие в блохастом грязненьке!
Идите!
Понедельники и вторники
окрасим кровью в праздники!
Пускай земле под ножами припомнится,
кого хотела опошлить!

Земле,
обжиревшей, как любовница,
которую вылюбил Ротшильд!
Чтоб флаги трепались в горячке пальбы,
как у каждого порядочного праздника —
выше вздымайте, фонарные столбы,
окровавленные туши лабазников.

Изругивался,
вымаливался,
резал,
лез за кем-то
вгрызаться в бока.

На небе, красный, как марсельеза,
вздрагивал, околевая, закат.

Уже сумашествие.

Ничего не будет.

Ночь придет,
перекусит
и съест.
Видите —
небо опять иудит
пригоршнью обгрызанных предательством звезд?

Пришла.
Пирует Мамаем,
задом на город насев.
Эту ночь глазами не проломаем,
черную, как Азеф!

Ежусь, зашвырнувшись в трактирные углы,
вином обливаю душу и скатерть
и вижу:
в углу — глаза круглы,—
глазами в сердце въелась богоматерь.
Чего одаривать по шаблону намалеванному
сиянием трактирную ораву!
Видишь — опять
голгофнику оплеванному
предпочитают Варавву?
Может быть, нарочно я
в человечьем месиве
лицом никого не новей.
Я,
может быть,
самый красивый
из всех твоих сыновей.
Дай им,
заплесневшим в радости,
скорой смерти времени,
чтоб стали дети, должные подрасти,
мальчики — отцы,
девочки — забеременели.
И новым рожденным дай обрасти
пытливой сединой волхвов,
и придут они —
и будут детей крестить
именами моих стихов.

Я, воспевающий машину и Англию,
может быть, просто,
в самом обыкновенном Евангелии
тринадцатый апостол.
И когда мой голос
похабно ухает —
от часа к часу,
целые сутки,
может быть, Иисус Христос нюхает
моей души незабудки.


4

Мария! Мария! Мария!
Пусти, Мария!
Я не могу на улицах!
Не хочешь?
Ждешь,
как щеки провалятся ямкою
попробованный всеми,
пресный,
я приду
и беззубо прошамкаю,
что сегодня я
«удивительно честный».
Мария,
видишь —
я уже начал сутулиться.

В улицах
люди жир продырявят в четырехэтажных зобах,
высунут глазки,
потертые в сорокгодовой таске,—
перехихикиваться,
что у меня в зубах
— опять!—
черствая булка вчерашней ласки.
Дождь обрыдал тротуары,
лужами сжатый жулик,
мокрый, лижет улиц забитый булыжником труп,
а на седых ресницах —
да!—
на ресницах морозных сосулек
слезы из глаз —
да!—
из опущенных глаз водосточных труб.
Всех пешеходов морда дождя обсосала,
а в экипажах лощился за жирным атлетом атлет;
лопались люди,
проевшись насквозь,
и сочилось сквозь трещины сало,
мутной рекой с экипажей стекала
вместе с иссосанной булкой
жевотина старых котлет.

Мария!
Как в зажиревшее ухо втиснуть им тихое слово?
Птица
побирается песней,
поет,
голодна и звонка,
а я человек, Мария,
простой,
выхарканный чахоточной ночью в грязную руку Пресни.
Мария, хочешь такого?
Пусти, Мария!
Судорогой пальцев зажму я железное горло звонка!

Мария!

Звереют улиц выгоны.
На шее ссадиной пальцы давки.

Открой!

Больно!

Видишь — натыканы
в глаза из дамских шляп булавки!

Пустила.

Детка!
Не бойся,
что у меня на шее воловьей
потноживотые женщины мокрой горою сидят,—
это сквозь жизнь я тащу
миллионы огромных чистых любовей
и миллион миллионов маленьких грязных любят.
Не бойся,
что снова,
в измены ненастье,
прильну я к тысячам хорошеньких лиц,—
«любящие Маяковского!»—
да ведь это ж династия
на сердце сумасшедшего восшедших цариц.
Мария, ближе!
В раздетом бесстыдстве,
в боящейся дрожи ли,
но дай твоих губ неисцветшую прелесть:
я с сердцем ни разу до мая не дожили,
а в прожитой жизни
лишь сотый апрель есть.
Мария!

Поэт сонеты поет Тиане,
а я —
весь из мяса,
человек весь —
тело твое просто прошу,
как просят христиане —
«хлеб наш насущный
даждь нам днесь».

Мария — дай!

Мария!
Имя твое я боюсь забыть,
как поэт боится забыть
какое-то
в муках ночей рожденное слово,
величием равное богу.
Тело твое
я буду беречь и любить,
как солдат,
обрубленный войною,
ненужный,
ничей,
бережет свою единственную ногу.
Мария —
не хочешь?
Не хочешь!

Ха!

Значит — опять
темно и понуро
сердце возьму,
слезами окапав,
нести,
как собака,
которая в конуру
несет
перееханную поездом лапу.
Кровью сердце дорогу радую,
липнет цветами у пыли кителя.
Тысячу раз опляшет Иродиадой
солнце землю —
голову Крестителя.
И когда мое количество лет
выпляшет до конца —
миллионом кровинок устелется след
к дому моего отца.

Вылезу
грязный (от ночевок в канавах),
стану бок о бок,
наклонюсь
и скажу ему наухо:
— Послушайте, господин бог!
Как вам не скушно
в облачный кисель
ежедневно обмакивать раздобревшие глаза?
Давайте — знаете —
устроимте карусель
на дереве изучения добра и зла!
Вездесущий, ты будешь в каждом шкапу,
и вина такие расставим постолу,
чтоб захотелось пройтись в ки-ка-пу
хмурому Петру Апостолу.
А в рае опять поселим Евочек:
прикажи,—
сегодня ночью ж
со всех бульваров красивейших девочек
я натащу тебе.
Хочешь?
Не хочешь?
Мотаешь головою, кудластый?
Супишь седую бровь?
Ты думаешь —
этот,
за тобою, крыластый,
знает, что такое любовь?
Я тоже ангел, я был им —
сахарным барашком выглядывал в глаз,
но больше не хочу дарить кобылам
из сервской муки изваянных ваз.
Всемогущий, ты выдумал пару рук,
сделал,
что у каждого есть голова,—
отчего ты не выдумал,
чтоб было без мук
целовать, целовать, целовать?!
Я думал — ты всесильный божище,
а ты недоучка, крохотный божик.
Видишь, я нагибаюсь,
из-за голенища
достаю сапожный ножик.
Крыластые прохвосты!
Жмитесь в раю!
Ерошьте перышки в испуганной тряске!
Я тебя, пропахшего ладаном, раскрою
отсюда до Аляски!

Пустите!

Меня не остановите.
Вру я,
в праве ли,
но я не могу быть спокойней.
Смотрите —
звезды опять обезглавили
и небо окровавили бойней!
Эй, вы!
Небо!
Снимите шляпу!
Я иду!

Глухо.

Вселенная спит,
положив на лапу
с клещами звезд огромное ухо.


"""

In [5]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# Сохраним обучающие данные в .txt файл 
train_path = 'train_dataset.txt'
with open(train_path, "wb") as f:
    f.write(text.encode('utf-8'))

# Создание датасета
train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=64, 
                            overwrite_cache=True)
  
# специальный класс который будет подавать в модель данные в нужном ей виде
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [6]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments( 
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,  
    gradient_accumulation_steps=16, 
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [7]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=100, training_loss=0.5286579132080078, metrics={'train_runtime': 135.524, 'train_samples_per_second': 59.768, 'train_steps_per_second': 0.738, 'total_flos': 235456782336000.0, 'train_loss': 0.5286579132080078, 'epoch': 89.0})

In [8]:
text = "Весна пришла "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.4,
                        top_k=50,
                        max_length=200,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Весна пришла 
и 
стала
духом больным —
и у меня, как
у кошки горя,
размягчилась спина,
в два года облезла,
одной голой
вышел в город,
из окон облез ли,
сам напялив — да уж, надо!
—
Ну, гляди! —
радовалась мне
в углу девчонка веснушка. —
Розы, розы!
Вот у нас
улица кофейней именуется:
пришла весна,
приняла
и посадила
цветы тюльпанные —
люди, народ, люди!

Девчонка — а весна — идет!
Она так красива!
И на одной из холмистых ям ям
живут ромашки белые, а на другой
уютнее всех стоят пузатые
горшки бань заграничных.

Всех влюбленных встречаешь,
когда на белом поле цветы


Выглядит очень даже неплохо))

In [9]:
text = "Послушайте! Ведь если "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.4,
                        top_k=50,
                        max_length=200,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Послушайте! Ведь если 
кто--то на сердце у меня рыдает из,—
и глаза у него полны страданий,
нужно сказать,—
«убог я до конца дней».
А богач 
поцелуем не накормит;
надо просить у ближнего 
по миллиону жалких ртов.
А человек—забулдыге весь —
я,
который ни разу не
погулял верхом,—
забудет Бога, как есть.
Приходите ко Мне всепрощения просить и поминовение.
Не будем больше кричать:
«Пришли! Пришли!» —
«Распни!»
Жмитесь пред богом как скиталец—
грязный, оборванный,
грязный из барака!
Помните: на рае
все однощие сидят:
в люке моются —
и односкатных коек два!
Голодные едят в бараний рог


In [11]:
# увеличиваем top_k с 50 до 100, убираем температуру, добавляем num_beams=6
text = "Послушайте! Ведь если "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=6,
                        top_k=100,
                        max_length=200,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Послушайте! Ведь если 
я 
буду 
грязным, как 
люди,
то 
я 
буду 
грязным, как 
люди,
которые 
бросают 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
грязные 
г


Получается значительно хуже

In [12]:
# возвращаем температуру, добавляем num_beams=6, запрещаем
text = "Послушайте! Ведь если "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.4,
                        num_beams=6,
                        top_k=100,
                        max_length=200,
                        no_repeat_ngram_size=2 
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Послушайте! Ведь если  вам  хочется,
чтоб  у  вашего  изголовья
обрубали  головы  —
вы  можете  себе это  позволить!

Ведь когда-нибудь
наступит день, и вы скажете:
«Я хочу, чтобы у меня были волосы, как у Иисуса!»
А может быть, вам все-таки хочется
улыбаться прохожим?
Вы, конечно, скажете, что у вас на лице
есть что-то такое, чего нет у других людей.
Но я не могу быть таким, каким вы на самом деле
виделися мне. И когда я говорю вам это, я
говорю не про себя, а про вас. Вы не можете не видеть
моего лица. Я вижу вас стоящим среди тысяч
тысяч людей, которые стоят и смотрят на меня
как на сумасшедшего, которого хотят придушить
многие из тех, кто стоит рядом с вами.


Получилось жутковато, но здорово! 

In [13]:
# пробую еще раз предыдущие настройки
text = "Пришла пора "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.4,
                        num_beams=6,
                        top_k=100,
                        max_length=200,
                        no_repeat_ngram_size=2 
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Пришла пора 
прибраться в доме,
в спальне, в кухне.
Вы не знаете, что такое —
любовь?
Это когда вы идете по лезвию ножа
и думаете:
«Господи!»
Любовь — это когда в глазах
обгорелых, обгоревших от любви глаз
не видно слез умиления!
Когда на лице твоем
потертая рубашка
изорвана в клочья, а на шее
бросается в лицо
горькая пощечина,—
любви не найти в этом доме
ни одного человека, который бы не заплакал от горя и не выл от обиды!—
этот дом не пропахнет ни розами, ни фиалками;
никто не узнает его в заплаканных глазах ни одной живой души!И ни один человек, кроме тебя, не скажет мне:«Боже, как я скучаю по тебе!»И никто не засмеется тебе,когда я скажу:


В конце немного сбилось, но результат мне очень нравится

In [14]:
# убираю top_k, добавляю num_return_sequences=4
text = "В конце концов "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.4,
                        num_beams=6,
                        max_length=200,
                        no_repeat_ngram_size=2,
                        num_return_sequences=4
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



В конце концов 
пришла я на площадь и сказала:
— Господа!
Я хочу, чтобы вы были богаче всех, кого я знаю.
А богатеют только те, у кого денег в кармане больше, чем у каждого,
у кого в доме есть хотя бы одна копейка
золотого унитаза, а в спальне — ни одной!—
каждый, кто хоть на миг усомнится в моей порядочности. 

Вы думаете: это я придумала?
Нет, на самом деле это придумала я сама. Я сама придумала, как надо жить, и как это здорово —
богаче всего жить в богатом доме, в двухкомнатной квартирке и в просторной квартире, которая выходит окнами на море. А теперь представьте, что у вас в голове — миллион миллионеров. И вы каждый день выходите на улицы, ютятся в дешевых отелях, ходите в дорогих ресторанах, любуетесь на роскошные


Под конец тоже немного сломалось, но результат неплохой

In [15]:
# убираю num_return_sequences, добавляю top_k=100, добавляю top_p=0.90 
text = "В конце концов "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.4,
                        num_beams=6,
                        max_length=200,
                        no_repeat_ngram_size=2,
                        top_k=100,
                        top_p=0.90 
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



В конце концов 
истинный христианин должен быть избран Богом.

И если кто-нибудь говорит:
«Я не хочу быть ничьим избранником»,
то он не имеет в виду никого другого, как одного из нас,
имевших много общего с Иисусом Христом и сделавших столько добра, сколько ни один не сделал бы ни одному из людей, не говоря уже о десятках тысяч других. 

Не может быть, чтобы у одного человека было больше сердца, чем у всех людей; но у каждого сердца есть много сердец, которые не в состоянии вместить в себя ни одного, и поэтому они не вмещаются в одну человеческую душу;
а в душе каждого человека сердце — это та самая человеческая душа, которая вмещает все, что есть на земле, —
которая не нуждается ни в ком, кроме как в любящем ее сердце!
Любящий душу свою и любящий ее сына, любит и брата своего, потому что он одно есть сердце


Кажется, получилось не очень хорошо по сравнению с предыдущими попытками

### Задание  2 (2 балла)

Ответьте на следующие вопросы:

1) В каких статья были представлены GPT-1, GPT-2, GPT-3?

2) Как собирался обучающий корпус для GPT-3? Каким образом создатели старались обеспечить высокое качество текстов в обучающей выборке?

1) 
- GPT-1: "Improving Language Understanding by Generative Pre-Training", 2018 (https://openai.com/research/language-unsupervised paper + code)
- GPT-2: paper "Language Models are Unsupervised Multitask Learners", 2019 (https://openai.com/research/better-language-models paper + code)
- GPT-3: "Language Models are Few-Shot Learners", 2020 (https://openai.com/research/language-models-are-few-shot-learners paper)

2) Основная часть обучающего корпуса - датасет Common Crawl, который содержит примерно 410 млрд byte-pair-encoded токенов (составляет 60% от всего корпуса).
Чтобы улучшить качество датасетов, авторы сначала отфильтровали датасет CommonCrawl на основе сходства со справочными датасетами высокого качества. Далее авторы осуществлили дедупликацию (fuzzy document deduplication), чтобы предотвратить избыточность (redundancy) и не допустить переобучения. Затем авторы расширили и разнообразили датасет Common Crawl за счет добавления датасетов WebText2 (19 млрд токенов), Books1 (12 млрд), Books2 (55 млрд) и английскую Wikipedia (3 млрд). 